In [16]:
import pandas as pd

def xlsx_to_csv(file_path):
    # Ler o arquivo XLSX usando o pandas
    df = pd.read_excel(file_path)

    # Salvar o DataFrame em um arquivo CSV
    csv_file_path = file_path.replace('.xlsx', '.csv')
    df.to_csv(csv_file_path, index=False)

    print(f'Arquivo CSV salvo em: {csv_file_path}')

# Exemplo de uso
xlsx_file_path = 'Estação_PUC_São_Gabriel_2021_-_BH_-_Regap.xlsx'
xlsx_to_csv(xlsx_file_path)

Arquivo CSV salvo em: Estação_PUC_São_Gabriel_2021_-_BH_-_Regap.csv


In [39]:
import os

def preencher_espacos_vazios_com_mediana(file_path):
    # Ler o arquivo CSV usando o pandas
    df = pd.read_csv(file_path)

    # Calcular a mediana das colunas PM10, NO2 e "PM 2,5"
    median_pm10 = df['PM10'].median()
    median_no2 = df['NO2'].median()
    median_pm25 = df['PM 2,5'].median()

    # Preencher espaços vazios nas colunas PM10, NO2 e "PM 2,5" com a mediana
    df['PM10'].fillna(median_pm10, inplace=True)
    df['NO2'].fillna(median_no2, inplace=True)
    df['PM 2,5'].fillna(median_pm25, inplace=True)

    if os.path.exists(file_path):
        os.remove(file_path)
        
    # Salvar o DataFrame atualizado em um novo arquivo CSV
    csv_file_path = file_path.replace('.csv', '_preenchido.csv')
    df.to_csv(csv_file_path, index=False)

    print(f'Arquivo CSV com espaços vazios preenchidos pela mediana salvo em: {csv_file_path}')

# Exemplo de uso
csv_file_path = 'Betim_Estação_Petrovale.csv'
preencher_espacos_vazios_com_mediana(csv_file_path)


Arquivo CSV com espaços vazios preenchidos pela mediana salvo em: Betim_Estação_Petrovale_preenchido.csv


In [ ]:
import numpy as np

def preencher_segundos_faltantes(file_path, columns_with_seconds):
    # Ler o arquivo CSV usando o pandas
    df = pd.read_csv(file_path)

    # Verificar colunas com segundos
    for column in columns_with_seconds:
        # Converter a coluna para o tipo datetime
        df[column] = pd.to_datetime(df[column], errors='coerce')

        # Obter os índices dos valores nulos na coluna
        null_indices = df[df[column].isnull()].index

        # Preencher os segundos faltantes com 0
        df[column] = df[column].fillna(pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:00'))

    # Excluir o arquivo anterior, se existir
    if os.path.exists(file_path):
        os.remove(file_path)

    # Salvar o DataFrame atualizado em um novo arquivo CSV
    csv_file_path = file_path.replace('_preenchido.csv', '.csv')
    df.to_csv(csv_file_path, index=False)

    print(f'Arquivo CSV com segundos faltantes preenchidos salvo em: {csv_file_path}')
    
# Exemplo de uso
csv_file_path = 'Estação_Veneza_2021_-_Ipatinga_-_Usiminas_preenchido_preenchido.csv'
preencher_espacos_vazios_com_mediana(csv_file_path)


In [40]:
import pandas as pd
import os

def criar_arquivos_agregados():
    # Caminho da pasta de origem e destino (mesmo caminho do código)
    pasta_origem = '.'
    pasta_destino = '.'

    # Lista os arquivos na pasta de origem
    arquivos = os.listdir(pasta_origem)

    # Exclui os diretórios específicos da lista de arquivos
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('.git')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('.ipynb_checkpoints')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('geo.rar')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('Geoespaciais.ipynb')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('estações.csv')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('bkp.rar')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('geo_2.rar')]
    # Cria um DataFrame vazio para cada arquivo
    dfs = []
    for arquivo in arquivos:
        arquivo_path = os.path.join(pasta_origem, arquivo)
        df = pd.read_csv(arquivo_path, encoding='latin-1')
        dfs.append(df)

    # Concatena os DataFrames na horizontal
    df_concatenado = pd.concat(dfs, axis=1)

    # Cria os três arquivos CSV com as colunas agregadas
    colunas = ['PM10', 'NO2', 'PM 2,5']
    for coluna in colunas:
        colunas_selecionadas = [f'{coluna}' for i in range(len(arquivos))]
        df_selecionado = df_concatenado[colunas_selecionadas]
        arquivo_saida = os.path.join(pasta_destino, f'{coluna}.csv')
        df_selecionado.to_csv(arquivo_saida, index=False, float_format='%.3f')  # Limita a 3 casas decimais

        print(f'Arquivo {arquivo_saida} criado com sucesso.')

# Exemplo de uso
criar_arquivos_agregados()


Arquivo .\PM10.csv criado com sucesso.
Arquivo .\NO2.csv criado com sucesso.
Arquivo .\PM 2,5.csv criado com sucesso.


In [43]:
import pandas as pd
arquivo_path = 'NO2.csv'
# Carregar o arquivo CSV
df = pd.read_csv(arquivo_path)

# Obter o índice da coluna "PM10.12"
indice_inicio_remocao = df.columns.tolist().index('NO2.12')

# Remover as colunas a partir de "PM10.12"
df = df.iloc[:, :indice_inicio_remocao]
if os.path.exists(arquivo_path):
    os.remove(arquivo_path)

# Salvar o DataFrame modificado em um novo arquivo CSV
df.to_csv(arquivo_path, index=False)

In [46]:
import pandas as pd

arquivo_path = 'PM 2,5.csv'
# Carregar o arquivo de origem
df_origem = pd.read_csv('Belo_Horizonte_Estação_Centro_Av_Contorno_preenchido.csv')

# Extrair a coluna "DATA"
coluna_data = df_origem['DATA']

# Carregar o arquivo de destino
df_destino = pd.read_csv(arquivo_path)

# Inserir a coluna "DATA" como a primeira coluna
df_destino.insert(0, 'DATA', coluna_data)

if os.path.exists(arquivo_path):
    os.remove(arquivo_path)

# Salvar o DataFrame modificado em um novo arquivo CSV
df_destino.to_csv(arquivo_path, index=False)


In [52]:
import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv('PM 2,5.csv')

df_melt = df.melt(id_vars =['DATA'])

df_melt.to_csv('PM2,5_quase.csv', index=False)

In [58]:
import pandas as pd

# Carregar o arquivo CSV
df1 = pd.read_csv('NO2_quase.csv')
df2 = pd.read_csv('estações.csv')

merged_df = pd.merge(df1, df2, left_on='variable', right_on='estacao')
merged_df = merged_df.drop('estacao', axis=1)

merged_df.to_csv('NO2_dalhe.csv', index=False)

df2

,estacao,longitude,latitude
0,Belo_Horizonte_Estacao_Delegacia_Amazonas,-43.998890,-19.940224
1,Belo_Horizonte_Estacao_Centro_Av_Contorno,-43.935693,-19.915952
2,Belo_Horizonte_Estacao_PUC_Barreiro,-44.026449,-19.977216
3,Belo_Horizonte_Estacao_PUC_Sao_Gabriel,-43.918994,-19.859372
4,Betim_Estacao_Centro_Administrativo,-44.212949,-19.967827
5,Betim_Estacao_Alterosa,-44.157424,-19.944506
6,Betim_Estacao_Petrovale,-44.112410,-19.994176
7,Paracatu,-46.886264,-17.205726
8,Ipatiga_Estacao_Bom_Retiro,-42.557184,-19.511989
9,Ipatiga_Estacao_Cariru,-42.529372,-19.492132


In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from ipyleaflet import Map, Heatmap, Marker, basemaps, WidgetControl
from ipywidgets import interact, IntSlider, HTML, Output

# Carregar os dados do CSV para um DataFrame pandas
df = pd.read_csv('PM10_dalhe.csv')

# Converte a coluna 'DATA' para o tipo datetime
df['DATA'] = pd.to_datetime(df['DATA'])

# Agrupa os dados por mês
df['month'] = df['DATA'].dt.to_period('M')
grouped = df.groupby('month')

# Calcula as médias dos valores por mês
means = grouped['value'].mean()

# Cria um widget de controle deslizante
slider = IntSlider(min=0, max=len(grouped.groups.keys())-1, step=1)

# Cria um mapa centrado no primeiro ponto do mês inicial
initial_month = next(iter(grouped.groups.keys()))
initial_data = grouped.get_group(initial_month)[['latitude', 'longitude', 'value']]
center = [initial_data.iloc[0]['latitude'], initial_data.iloc[0]['longitude']]
m = Map(center=center, zoom=12)

# Cria uma camada de heatmap para o mês inicial com cores mais sensíveis
heatmap = Heatmap(locations=initial_data.values.tolist(), radius=50, max=100, max_opacity=1.0)
m.add_layer(heatmap)

# Texto para exibir o mês e o ano atual
current_date = grouped.get_group(initial_month)['DATA'].iloc[0].strftime("%B %Y")
html = HTML(value=f"<h4>{current_date}</h4>")
control = WidgetControl(widget=html, position='topright')
m.add_control(control)

# Gráfico de linha para exibir os valores médios ao longo dos meses
output = Output()

# Criação do gráfico
plt.figure(figsize=(10, 5))
months = [str(month) for month in list(grouped.groups.keys())]
values = [means[month] for month in list(grouped.groups.keys())]
plt.plot(months, values, marker='o')
plt.xlabel('Mês')
plt.ylabel('Valor Médio')
plt.title('Valores Médios por Mês')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

# Função para atualizar o mapa e o texto com o mês selecionado
def update_heatmap(change):
    month = list(grouped.groups.keys())[change['new']]
    data = grouped.get_group(month)[['latitude', 'longitude', 'value']]

    # Obtém a média do valor para o mês selecionado
    mean_value = means[month]

    heatmap.locations = data.values.tolist()

    # Define o centro do mapa como as coordenadas do primeiro ponto do mês selecionado
    #m.center = [data.iloc[0]['latitude'], data.iloc[0]['longitude']]

    # Exibe a média dos valores no mapa
    m.add_layer(Marker(location=(data.iloc[0]['latitude'], data.iloc[0]['longitude']),
                       title=f'Média: {mean_value:.2f}', rise_on_hover=True))

    # Atualiza o texto com o mês e o ano atual
    current_date = grouped.get_group(month)['DATA'].iloc[0].strftime("%B %Y")
    html.value = f"<h4>{current_date}</h4>"

slider.observe(update_heatmap, 'value')

# Exibir o mapa e o controle deslizante
display(m)
display(slider)

# Exibir o gráfico
display(plt.gcf())
display(output)

In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

# Carregar os dados do CSV para um DataFrame pandas
df = pd.read_csv('PM10_dalhe.csv')

# Converte a coluna 'DATA' para o tipo datetime
df['DATA'] = pd.to_datetime(df['DATA'])

# Cria uma lista de cidades
cidades = ['Belo_Horizonte', 'Betim', 'Ipatinga', 'Paracatu']

# Criação do gráfico
plt.figure(figsize=(10, 5))

for cidade in cidades:

    # Seleciona apenas os dados para a cidade atual
    df_cidade = df[df['variable'].str.contains(cidade)].copy()

    # Agrupa os dados por mês
    df_cidade['month'] = df_cidade['DATA'].dt.to_period('M')
    grouped = df_cidade.groupby('month')

    # Calcula as médias dos valores por mês
    means = grouped['value'].mean()

    months = [str(month) for month in list(grouped.groups.keys())]
    values = [means[month] for month in list(grouped.groups.keys())]
    
    # Adiciona a linha para a cidade atual no gráfico
    plt.plot(months, values, marker='o', label=cidade.replace('_', ' '))  # Substitui '_' por ' '

# Ajuste os rótulos e legendas do gráfico
plt.xlabel('Mês')
plt.ylabel('Valor Médio')
plt.title('Valores Médios por Mês')
plt.xticks(rotation=45)
plt.legend(loc='best')  # Adiciona a legenda ao gráfico
plt.grid(True)
plt.tight_layout()

# Exibir o gráfico
plt.show()
